# ശ്രദ്ധാ യന്ത്രങ്ങളും ട്രാൻസ്ഫോർമറുകളും

പുനരാവർത്തന ശൃംഖലകളുടെ ഒരു പ്രധാന ദോഷം, ഒരു ശൃംഖലയിൽ ഉള്ള എല്ലാ വാക്കുകളും ഫലത്തിൽ ഒരുപോലെ സ്വാധീനം ചെലുത്തുന്നതാണ്. ഇത് നാമനിർദ്ദേശം തിരിച്ചറിയൽ, യന്ത്രഭാഷാന്തരം പോലുള്ള ശൃംഖല-തെ shൃംഖല പ്രവർത്തനങ്ങളിൽ സാധാരണ LSTM എൻകോഡർ-ഡികോഡർ മോഡലുകളുടെ പ്രകടനത്തെ കുറച്ചുകുറയിക്കുന്നു. യാഥാർത്ഥ്യത്തിൽ, ഇൻപുട്ട് ശൃംഖലയിൽ ചില പ്രത്യേക വാക്കുകൾ മറ്റുള്ളവയെക്കാൾ പരമ്പരാഗത ഔട്ട്പുട്ടുകളിൽ കൂടുതൽ സ്വാധീനം ചെലുത്താറുണ്ട്.

യന്ത്രഭാഷാന്തരം പോലുള്ള ശൃംഖല-തെ shൃംഖല മോഡൽ പരിഗണിക്കാം. ഇത് രണ്ട് പുനരാവർത്തന ശൃംഖലകളാൽ നടപ്പിലാക്കപ്പെടുന്നു, അവയിൽ ഒന്നായ **എൻകോഡർ** ഇൻപുട്ട് ശൃംഖലയെ ഹിഡൻ സ്റ്റേറ്റിലേക്ക് ചുരുക്കും, മറ്റൊന്ന്, **ഡികോഡർ**, ഈ ഹിഡൻ സ്റ്റേറ്റ് അന്വർത്ഥമായി വിവർത്തനം ചെയ്ത ഫലമായി തുറക്കും. ഈ സമീപനത്തിലെ പ്രശ്നം, ശൃംഖലയുടെ അന്തിമ അവസ്ഥ ഒരു വാക്യത്തിന്റെ തുടക്കം ഓർക്കാൻ ബുദ്ധിമുട്ട് അനുഭവപ്പെടുന്നതാണ്, അതിനാൽ ദീർഘ വാക്യങ്ങളിൽ മോഡലിന്റെ ഗുണമേന്മ കുറയുന്നു.

**ശ്രദ്ധാ യന്ത്രങ്ങൾ** RNN-ന്റെ ഓരോ ഔട്ട്പുട്ട് പ്രവചനത്തിലും ഓരോ ഇൻപുട്ട് വെക്ടറിന്റെ സാന്ദർഭിക സ്വാധീനം തൂക്കമിടാനുള്ള മാർഗം നൽകുന്നു. ഇത് നടപ്പിലാക്കുന്നത് ഇൻപുട്ട് RNN-ന്റെ ഇടനിലവസ്ഥകളും ഔട്ട്പുട്ട് RNN-ന്റെ ഇടനിലവസ്ഥകളും തമ്മിൽ ഷോർട്ട്കട്ടുകൾ സൃഷ്ടിച്ച് ആണ്. ഈ രീതിയിൽ, ഔട്ട്പുട്ട് ചിഹ്നം $y_t$ സൃഷ്ടിക്കുമ്പോൾ, വ്യത്യസ്ത തൂക്കം കോഫിഷ്യന്റുകളായ $\alpha_{t,i}$ ഉപയോഗിച്ച് എല്ലാ ഇൻപുട്ട് ഹിഡൻ സ്റ്റേറ്റുകളും $h_i$ പരിഗണിക്കും.

![Image showing an encoder/decoder model with an additive attention layer](../../../../../translated_images/ml/encoder-decoder-attention.7a726296894fb567.webp)
*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) ലെ ആഡിറ്റീവ് ശ്രദ്ധാ യന്ത്രം ഉൾപ്പെടുത്തിയ എൻകോഡർ-ഡികോഡർ മോഡൽ, [ഈ ബ്ലോഗ് പോസ്റ്റ്](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html) നിന്നെടുത്തത്]*

ശ്രദ്ധാ മാട്രിക്സ് $\{\alpha_{i,j}\}$ ഒരു പ്രത്യേക ഔട്ട്പുട്ട് വാക്കിന്റെ സൃഷ്ടിയിൽ ചില ഇൻപുട്ട് വാക്കുകൾ എത്രമാത്രം പങ്കുവഹിക്കുന്നുവെന്ന് പ്രതിനിധീകരിക്കും. താഴെ ഒരു ഉദാഹരണ മാട്രിക്സ് കാണിക്കുന്നു:

![Image showing a sample alignment found by RNNsearch-50, taken from Bahdanau - arviz.org](../../../../../translated_images/ml/bahdanau-fig3.09ba2d37f202a6af.webp)

*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (ചിത്രം 3) നിന്നെടുത്ത ചിത്രം]*

ശ്രദ്ധാ യന്ത്രങ്ങൾ പ്രകൃതിഭാഷാ പ്രോസസ്സിംഗിലെ നിലവിലെ അല്ലെങ്കിൽ അടുത്തകാലത്തെ ഏറ്റവും മികച്ച പ്രകടനത്തിന് വലിയ പങ്ക് വഹിക്കുന്നു. എന്നാൽ ശ്രദ്ധ ചേർക്കുന്നത് മോഡൽ പാരാമീറ്ററുകളുടെ എണ്ണം വളരെ വർദ്ധിപ്പിക്കുന്നതിനാൽ RNN-കളിൽ സ്കെയിലിംഗ് പ്രശ്നങ്ങൾ ഉണ്ടായി. RNN-കളുടെ സ്കെയിലിംഗിന്റെ പ്രധാന തടസ്സം, മോഡലുകളുടെ പുനരാവർത്തന സ്വഭാവം പരിശീലനം ബാച്ച് ചെയ്യാനും പാരലലൈസ് ചെയ്യാനും ബുദ്ധിമുട്ട് സൃഷ്ടിക്കുന്നു. RNN-യിൽ ഒരു ശൃംഖലയുടെ ഓരോ ഘടകവും പരമ്പരാഗത ക്രമത്തിൽ പ്രോസസ്സ് ചെയ്യേണ്ടതുണ്ടെന്നത് പാരലലൈസേഷൻ എളുപ്പമല്ല.

ശ്രദ്ധാ യന്ത്രങ്ങളുടെ സ്വീകരണവും ഈ തടസ്സവും ചേർന്ന് ഇന്ന് നാം അറിയുന്ന BERT മുതൽ OpenGPT3 വരെ ഉള്ള സ്റ്റേറ്റ് ഓഫ് ദി ആർട്ട് ട്രാൻസ്ഫോർമർ മോഡലുകളുടെ സൃഷ്ടിക്ക് വഴിതെളിച്ചു.

## ട്രാൻസ്ഫോർമർ മോഡലുകൾ

മുൻപ് ചെയ്ത ഓരോ പ്രവചനത്തിന്റെ സാന്ദർഭം അടുത്ത മൂല്യനിർണയ ഘട്ടത്തിലേക്ക് മുന്നോട്ട് അയക്കുന്നതിന് പകരം, **ട്രാൻസ്ഫോർമർ മോഡലുകൾ** **സ്ഥാനം സൂചിപ്പിക്കുന്ന എൻകോഡിംഗുകളും** ശ്രദ്ധയും ഉപയോഗിച്ച് നൽകിയ ടെക്സ്റ്റ് വിൻഡോയിൽ ഉള്ള ഇൻപുട്ടിന്റെ സാന്ദർഭം പിടിച്ചുപറ്റുന്നു. താഴെയുള്ള ചിത്രം സ്ഥാനം സൂചിപ്പിക്കുന്ന എൻകോഡിംഗുകളും ശ്രദ്ധയും ഉപയോഗിച്ച് ഒരു വിൻഡോയിൽ ഉള്ള സാന്ദർഭം എങ്ങനെ പിടിച്ചുപറ്റാമെന്ന് കാണിക്കുന്നു.

![Animated GIF showing how the evaluations are performed in transformer models.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

ഓരോ ഇൻപുട്ട് സ്ഥാനം സ്വതന്ത്രമായി ഓരോ ഔട്ട്പുട്ട് സ്ഥാനത്തേക്കും മാപ്പ് ചെയ്യപ്പെടുന്നതിനാൽ, ട്രാൻസ്ഫോർമറുകൾ RNN-കളേക്കാൾ മികച്ച പാരലലൈസേഷൻ സാധ്യമാക്കുന്നു, ഇത് വലിയതും കൂടുതൽ പ്രകടനക്ഷമവുമായ ഭാഷാ മോഡലുകൾ സൃഷ്ടിക്കാൻ സഹായിക്കുന്നു. ഓരോ ശ്രദ്ധാ തലയും വാക്കുകൾ തമ്മിലുള്ള വ്യത്യസ്ത ബന്ധങ്ങൾ പഠിക്കാൻ ഉപയോഗിക്കാം, ഇത് താഴെ വരുന്ന പ്രകൃതിഭാഷാ പ്രോസസ്സിംഗ് പ്രവർത്തനങ്ങൾ മെച്ചപ്പെടുത്തുന്നു.

**BERT** (Bidirectional Encoder Representations from Transformers) വളരെ വലിയ, 12 ലെയറുള്ള *BERT-base*നും 24 ലെയറുള്ള *BERT-large*നും ഉള്ള മൾട്ടി ലെയർ ട്രാൻസ്ഫോർമർ നെറ്റ്‌വർക്കാണ്. മോഡൽ ആദ്യം വലിയ ടെക്സ്റ്റ് കോർപ്പസ് (വിക്കിപീഡിയ + പുസ്തകങ്ങൾ) ഉപയോഗിച്ച് സ്വയംപരിശീലനത്തിലൂടെ (വാക്യത്തിലെ മറച്ചുവച്ച വാക്കുകൾ പ്രവചിച്ച്) പ്രീ-ട്രെയിൻ ചെയ്യപ്പെടുന്നു. പ്രീ-ട്രെയിനിംഗിൽ മോഡൽ ഭാഷാ ബോധം ഗഹനമായി ഉൾക്കൊള്ളുന്നു, പിന്നീട് ഇത് മറ്റ് ഡാറ്റാസെറ്റുകൾ ഉപയോഗിച്ച് ഫൈൻ ട്യൂണിങ്ങിലൂടെ പ്രയോജനപ്പെടുത്താം. ഈ പ്രക്രിയ **ട്രാൻസ്ഫർ ലേണിംഗ്** എന്ന് വിളിക്കുന്നു.

![picture from http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/ml/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

BERT, DistilBERT, BigBird, OpenGPT3 തുടങ്ങിയ നിരവധി ട്രാൻസ്ഫോർമർ ആർക്കിടെക്ചറുകളുടെ വ്യത്യാസങ്ങൾ ഉണ്ട്, ഇവ ഫൈൻ ട്യൂൺ ചെയ്യാവുന്നതാണ്. [HuggingFace പാക്കേജ്](https://github.com/huggingface/) PyTorch ഉപയോഗിച്ച് ഈ ആർക്കിടെക്ചറികളുടെ പരിശീലനത്തിനുള്ള റിപോസിറ്ററി നൽകുന്നു.

## ടെക്സ്റ്റ് ക്ലാസിഫിക്കേഷനിൽ BERT ഉപയോഗിക്കൽ

നമ്മുടെ പരമ്പരാഗത പ്രവർത്തനം: ശൃംഖല ക്ലാസിഫിക്കേഷൻ പരിഹരിക്കാൻ പ്രീ-ട്രെയിൻ ചെയ്ത BERT മോഡൽ എങ്ങനെ ഉപയോഗിക്കാമെന്ന് നോക്കാം. നാം നമ്മുടെ യഥാർത്ഥ AG News ഡാറ്റാസെറ്റ് ക്ലാസിഫൈ ചെയ്യും.

ആദ്യം, HuggingFace ലൈബ്രറിയും നമ്മുടെ ഡാറ്റാസെറ്റും ലോഡ് ചെയ്യാം:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


നാം പ്രീ-ട്രെയിൻ ചെയ്ത BERT മോഡൽ ഉപയോഗിക്കാനിരിക്കുന്നതിനാൽ, പ്രത്യേക ടോക്കനൈസർ ഉപയോഗിക്കേണ്ടതുണ്ട്. ആദ്യം, പ്രീ-ട്രെയിൻ ചെയ്ത BERT മോഡലുമായി ബന്ധപ്പെട്ട ടോക്കനൈസർ ലോഡ് ചെയ്യാം.

HuggingFace ലൈബ്രറിയിൽ പ്രീ-ട്രെയിൻ ചെയ്ത മോഡലുകളുടെ ഒരു റിപോസിറ്ററി ഉണ്ട്, അവയുടെ പേരുകൾ `from_pretrained` ഫംഗ്ഷനുകൾക്ക് ആർഗ്യുമെന്റായി നൽകുന്നതിലൂടെ നിങ്ങൾക്ക് അവ ഉപയോഗിക്കാം. മോഡലിനുള്ള എല്ലാ ആവശ്യമായ ബൈനറി ഫയലുകളും സ്വയം ഡൗൺലോഡ് ചെയ്യും.

എങ്കിലും, ചിലപ്പോൾ നിങ്ങൾക്ക് നിങ്ങളുടെ സ്വന്തം മോഡലുകൾ ലോഡ് ചെയ്യേണ്ടി വരും, അപ്പോൾ ടോക്കനൈസറിന്റെ പാരാമീറ്ററുകൾ, മോഡൽ പാരാമീറ്ററുകളുള്ള `config.json` ഫയൽ, ബൈനറി വെയ്റ്റുകൾ എന്നിവ ഉൾപ്പെടെയുള്ള എല്ലാ ബന്ധപ്പെട്ട ഫയലുകളും അടങ്ങിയ ഡയറക്ടറി നിങ്ങൾക്ക് വ്യക്തമാക്കാം.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` ഒബ്ജക്റ്റിൽ `encode` ഫംഗ്ഷൻ ഉൾപ്പെടുന്നു, ഇത് ടെക്സ്റ്റ് എൻകോഡ് ചെയ്യാൻ നേരിട്ട് ഉപയോഗിക്കാം:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

അപ്പോൾ, പരിശീലനത്തിനിടെ ഡാറ്റ ആക്‌സസ് ചെയ്യാൻ ഉപയോഗിക്കുന്ന ഇറ്ററേറ്ററുകൾ സൃഷ്ടിക്കാം. BERT അതിന്റെ സ്വന്തം എൻകോഡിംഗ് ഫംഗ്ഷൻ ഉപയോഗിക്കുന്നതിനാൽ, മുമ്പ് നാം നിർവചിച്ച `padify` പോലുള്ള ഒരു പാഡിംഗ് ഫംഗ്ഷൻ നിർവചിക്കേണ്ടതുണ്ട്:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

നമ്മുടെ കേസിൽ, നാം `bert-base-uncased` എന്ന പേരിലുള്ള പ്രീ-ട്രെയിൻ ചെയ്ത BERT മോഡൽ ഉപയോഗിക്കും. `BertForSequenceClassification` പാക്കേജ് ഉപയോഗിച്ച് മോഡൽ ലോഡ് ചെയ്യാം. ഇത് നമ്മുടെ മോഡലിന് ക്ലാസിഫിക്കേഷനായി ആവശ്യമായ ആർക്കിടെക്ചർ, ഫൈനൽ ക്ലാസിഫയർ ഉൾപ്പെടെ, ഇതിനകം ഉണ്ടെന്ന് ഉറപ്പാക്കുന്നു. ഫൈനൽ ക്ലാസിഫയറിന്റെ വെയ്റ്റുകൾ ഇൻഷിയലൈസ് ചെയ്തിട്ടില്ലെന്നു കാണിക്കുന്ന മുന്നറിയിപ്പ് സന്ദേശം നിങ്ങൾക്ക് കാണാം, മോഡലിന് പ്രീ-ട്രെയിനിംഗ് ആവശ്യമുണ്ടെന്ന് പറയുന്നതും - അത് പൂർണ്ണമായും ശരിയാണ്, കാരണം നാം ചെയ്യാൻ പോകുന്നത് അതേയാണ്!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

ഇപ്പോൾ നാം പരിശീലനം ആരംഭിക്കാൻ തയ്യാറാണ്! BERT ഇതിനകം പ്രീ-ട്രെയിൻ ചെയ്തതിനാൽ, പ്രാരംഭ ഭാരങ്ങൾ നശിപ്പിക്കാതിരിക്കാൻ ചെറിയ ലേണിംഗ് റേറ്റ് ഉപയോഗിച്ച് തുടങ്ങാൻ ആഗ്രഹിക്കുന്നു.

എല്ലാ കഠിനപ്രവർത്തനവും `BertForSequenceClassification` മോഡലാണ് നിർവഹിക്കുന്നത്. പരിശീലന ഡാറ്റയിൽ മോഡൽ വിളിക്കുമ്പോൾ, അത് ഇൻപുട്ട് മിനിബാച്ചിനുള്ള നഷ്ടവും നെറ്റ്‌വർക്ക് ഔട്ട്പുട്ടും നൽകുന്നു. പാരാമീറ്റർ ഒപ്റ്റിമൈസേഷനായി നഷ്ടം ഉപയോഗിക്കുന്നു (`loss.backward()` ബാക്ക്വേഡ് പാസ് നടത്തുന്നു), പരിശീലന കൃത്യത കണക്കാക്കാൻ `out` ഉപയോഗിക്കുന്നു, ലഭിച്ച ലേബലുകൾ `labs` (`argmax` ഉപയോഗിച്ച് കണക്കാക്കിയ) പ്രതീക്ഷിച്ച `labels`-നൊപ്പം താരതമ്യം ചെയ്ത്.

പ്രക്രിയ നിയന്ത്രിക്കാൻ, നാം പല ഇറ്ററേഷനുകളിലായി നഷ്ടവും കൃത്യതയും സമാഹരിച്ച്, ഓരോ `report_freq` പരിശീലന ചക്രത്തിനും പ്രിന്റ് ചെയ്യുന്നു.

ഈ പരിശീലനം വളരെ സമയം എടുക്കാൻ സാധ്യതയുള്ളതിനാൽ, നാം ഇറ്ററേഷനുകളുടെ എണ്ണം പരിമിതപ്പെടുത്തുന്നു.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


നിങ്ങൾക്ക് കാണാം (പ്രത്യേകിച്ച് നിങ്ങൾ ഇറ്ററേഷനുകളുടെ എണ്ണം വർദ്ധിപ്പിച്ച് മതിയായ സമയം കാത്തിരിക്കുമ്പോൾ) BERT ക്ലാസിഫിക്കേഷൻ നമുക്ക് നല്ല കൃത്യത നൽകുന്നു! അതിന്റെ കാരണം BERT ഭാഷയുടെ ഘടന വളരെ നന്നായി മനസ്സിലാക്കുന്നുണ്ടെന്നതാണ്, അതിനാൽ നമുക്ക് അവസാന ക്ലാസിഫയർ മാത്രം ഫൈൻ-ട്യൂൺ ചെയ്യേണ്ടതുണ്ട്. എന്നാൽ, BERT ഒരു വലിയ മോഡലായതിനാൽ, മുഴുവൻ പരിശീലന പ്രക്രിയയും ഏറെ സമയം എടുക്കുന്നു, കൂടാതെ ഗണനശേഷി (GPU, സാധാരണയായി ഒന്നിലധികം) ആവശ്യമാണ്!

> **Note:** നമ്മുടെ ഉദാഹരണത്തിൽ, നാം ഏറ്റവും ചെറിയ പ്രീ-ട്രെയിൻ ചെയ്ത BERT മോഡലുകളിൽ ഒന്നാണ് ഉപയോഗിക്കുന്നത്. മികച്ച ഫലങ്ങൾ നൽകാൻ സാധ്യതയുള്ള വലിയ മോഡലുകളും ഉണ്ട്.


## മോഡൽ പ്രകടനം വിലയിരുത്തൽ

ഇപ്പോൾ നാം ടെസ്റ്റ് ഡാറ്റാസെറ്റിൽ നമ്മുടെ മോഡലിന്റെ പ്രകടനം വിലയിരുത്താം. വിലയിരുത്തൽ ലൂപ്പ് പരിശീലന ലൂപ്പിനോട് വളരെ സമാനമാണ്, പക്ഷേ `model.eval()` വിളിച്ച് മോഡൽ വിലയിരുത്തൽ മോഡിലേക്ക് മാറ്റാൻ മറക്കരുത്.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## പ്രധാനപ്പെട്ട കാര്യങ്ങൾ

ഈ യൂണിറ്റിൽ, **transformers** ലൈബ്രറിയിൽ നിന്നുള്ള മുൻകൂട്ടി പരിശീലിച്ച ഭാഷാ മോഡൽ എളുപ്പത്തിൽ എടുക്കുകയും അത് നമ്മുടെ ടെക്സ്റ്റ് ക്ലാസിഫിക്കേഷൻ ടാസ്ക്കിനായി അനുയോജ്യമായി മാറ്റുകയും ചെയ്യുന്നത് എങ്ങനെ എന്നത് കണ്ടു. അതുപോലെ, BERT മോഡലുകൾ എന്റിറ്റി എക്സ്ട്രാക്ഷൻ, ചോദ്യോത്തരങ്ങൾ, മറ്റ് NLP ടാസ്കുകൾ എന്നിവയ്ക്കും ഉപയോഗിക്കാം.

Transformer മോഡലുകൾ NLP-യിലെ ഏറ്റവും പുതിയ സാങ്കേതികവിദ്യ പ്രതിനിധീകരിക്കുന്നു, കൂടാതെ അധികം കേസുകളിൽ കസ്റ്റം NLP പരിഹാരങ്ങൾ നടപ്പിലാക്കുമ്പോൾ പരീക്ഷണം ആരംഭിക്കേണ്ട ആദ്യ പരിഹാരമായിരിക്കണം. എന്നിരുന്നാലും, ഈ മഡ്യൂളിൽ ചർച്ച ചെയ്ത റികറന്റ് ന്യൂറൽ നെറ്റ്‌വർക്കുകളുടെ അടിസ്ഥാന സിദ്ധാന്തങ്ങൾ മനസിലാക്കുന്നത്, നിങ്ങൾക്ക് ഉയർന്ന തലത്തിലുള്ള ന്യൂറൽ മോഡലുകൾ നിർമ്മിക്കാനാഗ്രഹിക്കുന്നുവെങ്കിൽ വളരെ പ്രധാനമാണ്.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റായ വിവരങ്ങൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖ അധികാരപരമായ ഉറവിടമായി കണക്കാക്കണം. നിർണായക വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
